In [ ]:
%pylab inline
style.use('http://johannesfeist.eu/misc/jf.mplstyle')
np.set_printoptions(linewidth=200)

In [ ]:
from qutip import *

In [ ]:
from numba import jit
%reload_ext Cython
%reload_ext fortranmagic

In [ ]:
from scipy.sparse import lil_matrix
N = 100000
H = lil_matrix((N,N))
H[range(N),range(N)] = -2.
H[range(N-1),range(1,N)] = 1
H[range(1,N),range(N-1)] = 1
nvals = 100*N
ii = random.randint(N,size=(2,nvals))
H[ii[0],ii[1]] = random.rand(nvals)
H = H.tocsr()
Hc = H.astype(np.complex128)

In [ ]:
phi0 = exp(-(arange(N)-15000)**2/(2*300**2)-1.5j*arange(N)) + exp(-(arange(N)-5000)**2/(2*50**2)+1j*arange(N))
phi0 /= norm(phi0)
phir = randn(N).astype(complex)
phir /= norm(phir)

In [ ]:
@jit(nopython=True,nogil=True)
def csr_matvec_numba(n_row,n_col,Ap,Aj,Ax,Xx,Yx):
    for i in range(n_row):
        val = Yx[i]
        for j in range(Ap[i],Ap[i+1]):
            val += Ax[j] * Xx[Aj[j]]
        Yx[i] = val

In [ ]:
%%fortran --opt "-O3 -finline-functions -fomit-frame-pointer -fno-strict-aliasing" --arch "-march=native"
subroutine csr_matvec_fort(n_row,Ap,Aj,Ax,Xx,a,Yx)
  integer, intent(in) :: n_row,Ap(:),Aj(:)
  real(8), intent(in) :: Ax(:), a
  complex(8), intent(in) :: Xx(:)
  complex(8), intent(inout) :: Yx(:)
  integer :: i, j
  complex(8) :: val
  do i = 1, n_row
    val = 0.
    do j = Ap(i)+1,Ap(i+1)
      val = val + Ax(j)*Xx(Aj(j)+1)
    end do
    Yx(i) = Yx(i) + a*val
  end do
end subroutine

In [ ]:
%%fortran --opt "-O3 -finline-functions -fomit-frame-pointer -fno-strict-aliasing" --arch "-march=native"
subroutine save_vecs(n_row,Ap,Aj,Ax)
  implicit none
  integer, intent(in) :: n_row,Ap(:),Aj(:)
  real(8), intent(in) :: Ax(:)
  write(501) n_row, size(Aj)
  write(502) Ap, Aj, Ax
  write(60,*) n_row, size(Aj)
  write(60,*) Ap(1:3)
  write(60,*) Aj(1:3)
  write(60,*) Ax(1:3)
  close(501)
  close(502)
  close(60)
end subroutine

In [ ]:
save_vecs(H.shape[0],H.indptr,H.indices,H.data)
!cat fort.60

In [ ]:
%%cython -a -c=-O3 -c=-march=native
import cython
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.embedsignature(True)
def csr_matvec_cy(size_t n_row, size_t n_col, int[::1] Ap, int[::1] Aj, double[::1] Ax,
                  double complex[::1] Xx, double a, double complex[::1] Yx):
    cdef:
        size_t i,j
        double complex val
    for i in range(n_row):
        val = 0. #Yx[i]
        for j in range(Ap[i],Ap[i+1]):
            val += Ax[j] * Xx[Aj[j]]
        Yx[i] = Yx[i] + a*val

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.embedsignature(True)
def csr_matvec_cyc(size_t n_row, size_t n_col, int[::1] Ap, int[::1] Aj, complex[::1] Ax, complex[::1] Xx, complex[::1] Yx):
    cdef:
        size_t i,j
        complex val
    for i in range(n_row):
        val = Yx[i]
        for j in range(Ap[i],Ap[i+1]):
            val += Ax[j] * Xx[Aj[j]]
        Yx[i] = val

In [ ]:
from qutip.cy.spmatfuncs import spmvpy
from scipy.sparse._sparsetools import csr_matvec

In [ ]:
phitest = H.dot(phi0)
phir *= 0.
csr_matvec(N,N,H.indptr,H.indices,H.data,phi0,phir)
print(norm(phitest-phir))

In [ ]:
phitest = H.dot(phi0)
def testfunc(f):
    import timeit
    global phir
    phir *= 0.
    f()
    print("%.1e"%norm((phitest-phir)/phitest),end=' ')
    t1 = timeit.Timer(f)
    print("%5.1f ms"%(t1.timeit(10)/10 * 1e3))
testfunc(lambda: csr_matvec(N,N,H.indptr,H.indices,H.data,phi0,phir))

In [ ]:
def testd(H,phi,Hphi):
    Hphi[:] = H.dot(phi)
testfunc(lambda: testd(H,phi0,phir))
testfunc(lambda: csr_matvec(N,N,H.indptr,H.indices,H.data,phi0,phir))
testfunc(lambda: csr_matvec_numba(N,N,H.indptr,H.indices,H.data,phi0,phir))
testfunc(lambda: csr_matvec_cy(N,N,H.indptr,H.indices,H.data,phi0,1.,phir))
testfunc(lambda: csr_matvec_cyc(N,N,Hc.indptr,Hc.indices,Hc.data,phi0,phir))
testfunc(lambda: csr_matvec_fort(N,H.indptr,H.indices,H.data,phi0,1.,phir))
testfunc(lambda: spmvpy(Hc.data,Hc.indices,Hc.indptr,phi0,1.,phir))